## Imports

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.data.hashs import compute_hash
from src.data.constants import ABS_FILE_PATH_ITEMS
from src.data.database import fetch_and_store_data, postgresql_command

## Table of Contents

1. [Data Collection](#data-collection)
2. [Data Validation](#data-validation)
3. [Preprocessing](#preprocessing)

## Data Collection

In [11]:
command = """SELECT time, data
FROM market_data
WHERE region_id = 0"""

results = postgresql_command(command, results=True)

Command executed successfully


In [13]:
print(results)

[(datetime.datetime(2022, 5, 23, 17, 42, 14, 877413), []), (datetime.datetime(2022, 5, 23, 17, 43, 39, 109944), {'2358': {'buy': {'weightedAverage': '28212.1660588', 'max': '48650.0', 'min': '0.03', 'stddev': '17484.3402462', 'median': '30020.0', 'volume': '65525.0', 'orderCount': '39', 'percentile': '48649.0835559'}, 'sell': {'weightedAverage': '103622.895317', 'max': '158500.0', 'min': '45000.0', 'stddev': '32934.4462974', 'median': '87965.0', 'volume': '38545.0', 'orderCount': '16', 'percentile': '47262.9394215'}}, '9842': {'buy': {'weightedAverage': '6018.90591754', 'max': '9901.0', 'min': '0.01', 'stddev': '4166.74857577', 'median': '7779.5', 'volume': '940695.0', 'orderCount': '114', 'percentile': '9846.81554064'}, 'sell': {'weightedAverage': '11050.5213137', 'max': '99980.0', 'min': '164.7', 'stddev': '11173.8518821', 'median': '10485.0', 'volume': '1231042.0', 'orderCount': '82', 'percentile': '9339.89511325'}}, '9846': {'buy': {'weightedAverage': '56982.7169199', 'max': '10310

In [12]:
[print(i[0]['2358']['buy']['weightedAverage']) for i in results if i[0] != []]

TypeError: 'datetime.datetime' object is not subscriptable

## Preprocessing